<center>
    <h1>Reinforcement Learning</h1>
    <h2>The Noughts and Crosses Challenge</h2>
</center>

There are several different areas in Machine Learning. One of these is <em>reinforcement learning</em>. This is often not as relevant to data science as the other main areas of ML, so we do not cover any reinforcement learning in our curriculum. This notebook is an introduction to the idea of reinforcement learning and a coding challenge as a stretch exercise for people confident in their Python skills.

For a brief introduction, and the inspiration for this notebook, see <a href="https://www.youtube.com/watch?v=R9c-_neaxeU">this video</a>. I strongly recommend you watch that video as well as reading this introduction.

<h3>
    Reinforcement learning: what's the big idea?
</h3>
Reinforcement learning is an approach to machine learning based on the way humans (and other animals) learn how to act in the world. When we take an action (e.g., eating some chocolate, or touching a hot stove) we receive a <em>reward</em> (such as a pleasant taste) or a <em>punishment</em> (such as a painful burn). Over time, we learn to repeat the actions which have tended to lead to rewards, and avoid the actions which lead to punishments. So reinforcement learning is about giving a computer program the ability to choose actions, giving it consistent rewards or punishments for those actions, and then training it over time to seek rewards and avoid punishments.

An area where this is particularly applicable is in <em>competitive games</em>, and reinforcement learning programs have become extremely successful at playing complex strategy games. In 2016, a system called AlphaGo, developed by Google using reinforcement learning techniques, played five games against Lee Sedol, one of the world's top players of the ancient strategy game Go. AlphaGo won 4 out of the 5 games - the first time a computer beat a world-class Go player without a handicap. Since then, Google have refined their techniques, and their newer AlphaGo Zero beat the earlier AlphaGo by 100 games to 0. There is a free documentary about AlphaGo <a href="https://www.youtube.com/watch?v=WXuK6gekU1Y">here</a>.

Go is a complicated game, so we will use a simpler example to introduce reinforcement learning: Noughts-and-crosses (aka Tic-Tac-Toe). In case anyone isn't familiar with this game, a brief summary:
Noughts-and-crosses is played on a 3x3 square grid. The two players each have a symbol (O for noughts, X for crosses). The players take turns to draw their symbol into one of the empty cells on the grid. If one player gets "three in a row" of their symbol - horizontally, vertically, or diagonally - that player wins and the game ends. As their are only 9 cells, after 9 moves, their will be no empty cells left; if this happens and no player has won, the game ends in a draw.

<h3>
    How does reinforcement learning actually work?
</h3>
The setup for a reinforcement learning is as follows. There is an <em>environment</em> which can be in a number of different <em>states</em>. For instance, in noughts-and-crosses, the environment is the board and the state of the environment is which cells have noughts in, which have crosses in, and which are still empty.

In each state, the <em>player</em> has a choice of <em>actions</em>. The environment then changes to a new state (or maybe the same one again, in some games) in response to the action. The way the environment changes might depend on factors outside the player's control, as well as the action chosen. For instance, in noughts-and-crosses, when I make a move I know exactly what state the board will be in after my move, but then the other player takes a turn, and I cannot control that. So the state when I next get to take an action depends partly on the previous actions I took, but also on factors outside my control.

At some point, the player is given <em>feedback</em>. This can happen after every action, or after several actions. The feedback can be positive (a reward), or negative (a punishment), and can have different strengths. It can also be neutral. In noughts-and-crosses, the most obvious way to do it is to give feedback at the end of each game (rather than after each turn), and to give positive feedback for a win, negative feedback for a defeat, and neutral feedback for a draw. You could always give the same "strength" of positive or negative feedback, or you could give stronger feedback the quicker the game ended, so that winning quickly has a bigger reward than winning slowly, and losing quickly has a bigger punishment than losing slowly. Other games with more complicated rules might have more complicated systems of feedback; e.g., in chess, winning the game should carry the highest reward, but capturing a piece might also have a small reward (maybe depending on the value of the piece).

After receiving feedback, the player adjusts their strategies (the <em>learning phase</em>) so that, in future, if it is faced with a state it has seen before, it is more likely to repeat an action that led to a reward in the past, and less likely to repeat an action that led to punishment. This can be done by coding probabilities for each possible action in each possible state, and then adjusting those probabilities up or down based on the feedback.

<h3>
    This notebook
</h3>
The aim of this notebook is to develop your Python skills by getting you to code a reinforcement learning algorithm for noughts-and-crosses from scratch. You are provided with two Python classes, NoughtsCrosses, and NoughtsCrossesMatch, as well as an outline for a third class, NoughtsCrossesPlayer. You should attempt to complete the NoughtsCrossesPlayer class to create a program that can play noughts-and-crosses and learn from experience.

You should start by running the first code cells to set up the NoughtsCrosses class. Then there is an explanation of how to use this with example code. Then there is the NoughtsCrossesMatch code cell, which you should run, and explanation and examples. Then you are given code to randomly select from several options with specified probabilities, as this will be useful for coding your player. Then you are given the NoughtsCrossesPlayer class to complete. Finally, there is a "tournament" cell which will pit two copies of your player against each other for several rounds and track the results. In noughts-and-crosses, if both players have perfect play, the game always ends in a draw, so this should eventually be the outcome of the tournament, after enough rounds have been played for the players to evolve perfect play.

In [1]:
# run this cell. No need to read it unless you're interested or you think there's a bug

class NoughtsCrosses:
    def __init__(self):
        self.board = [['', '', ''], ['', '', ''], ['', '', '']]
        self.is_game_over = False
        self.turn = 0 # number of completed turns

    def display_board(self):
        '''
        print the game board with some rough formatting
        '''
        for row in self.board.copy():
            print("-------------")
            for cell in row:
                if cell == '':
                    cell = ' '
                print(f"| {cell} ", end='')
            print("|")
        print("-------------")

    def check_rows(self):
        '''
        check if any player has won horizontally
        return winner's symbol, or None if no winner
        '''
        for row in self.board:
            if row[0] == row[1] and row[1] == row[2] and row[0] != '':
                return row[0]
        return None
    
    def check_cols(self):
        '''
        check if any player has won vertically
        return winner's symbol, or None if no winner
        '''
        for col in range(3):
            if self.board[0][col] == self.board[1][col] and self.board[1][col] == self.board[2][col] and self.board[0][col] != '':
                return self.board[0][col]
        return None
    
    def check_diags(self):
        '''
        check if any player has won diagonally
        return winner's symbol, or None if no winner
        '''
        if self.board[0][0] == self.board[1][1] and self.board[1][1] == self.board[2][2] and self.board[0][0] != '':
            return self.board[0][0]
        elif self.board[0][2] == self.board[1][1] and self.board[1][1] == self.board[2][0] and self.board[2][0] != '':
            return self.board[0][2]
        return None
        
    def check_game_over(self):
        '''
        check for winner. return winner's symbol, or 'D' if draw, or 'continue' if game still in progress
        '''
        rows, cols, diags = self.check_rows(), self.check_cols(), self.check_diags()
        if (rows, cols, diags) == (None, None, None):
            if self.turn >= 9:
                self.is_game_over = True
                return 'D'
            else:
                return 'continue'
        else:
            self.is_game_over = True
            if 'X' in (rows, cols, diags):
                return 'X'
            else: # must be Noughts
                return 'O'
    
    def make_move(self, row, col):
        '''
        play in cell (row, col) (0-indexed, so row and col can be 0, 1, or 2)
        will check_game_over and return from that
        if move is illegal (args out of bounds, or playing in a non-empty cell, or game is already over), returns 'F' (for 'Forfeit')
        '''
        out_of_bounds = not (row in (0, 1, 2) and col in (0, 1, 2))
        if self.is_game_over or out_of_bounds:
            self.is_game_over = True
            return 'F'
        elif self.board[row][col] != '': # needed to check out_of_bounds first to avoid IndexError
            self.is_game_over = True
            return 'F'
        # move is legal, so proceed:
        self.board[row][col] = ['O', 'X'][self.turn % 2]
        self.turn += 1
        return self.check_game_over()
    
    def reset(self):
        '''
        reset the game
        '''
        self.__init__()   

<h3>
    Explanation of code:
</h3>
The NoughtsCrosses class creates a noughts-and-crosses game. It has attributes for the turn number (0-indexed), whether the game is over or not, and the current state of the board (as a list of 3 lists, each of those lists being a row of 3 strings, which can be empty, 'X', or 'O'). It has methods to display the board, check for a winner or a draw, make a move, and reset the game (clear the board). <strong>The code assumes that noughts always plays first, and uses the turn number to determine the player: even turn number means noughts to play, odd turn number means crosses to play.</strong>

You can use the class as shown:

In [2]:
# instantiate
game = NoughtsCrosses()

In [3]:
# display board; should be blank initially
game.display_board()

# print turn number and whether game is over
print(game.turn, game.is_game_over)

# print internal representation of board
print(game.board)

-------------
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------
0 False
[['', '', ''], ['', '', ''], ['', '', '']]


In [4]:
print('Make a move:')

# make a move in the middle row, right-hand column
game.make_move(1, 2)

# display board; should now show 'O' in middle row, right-hand column
game.display_board()

# print turn number and whether game is over
print(game.turn, game.is_game_over)

# print internal representation of board
print(game.board)

Make a move:
-------------
|   |   |   |
-------------
|   |   | O |
-------------
|   |   |   |
-------------
1 False
[['', '', ''], ['', '', 'O'], ['', '', '']]


In [5]:
print('Make more moves:')

# make several more moves; note these return 'continue'
print(game.make_move(0, 0)) # crosses top left

print(game.make_move(2, 2)) # noughts bottom right

print(game.make_move(1, 1)) # crosses middle

print(game.make_move(0, 2)) # noughts top right, winning with right-hand vertical - now returns 'O' instead of None, as noughts has won

# display board; should now show completed game
game.display_board()

# print turn number and whether game is over
print(game.turn, game.is_game_over)

# print internal representation of board
print(game.board)

Make more moves:
continue
continue
continue
O
-------------
| X |   | O |
-------------
|   | X | O |
-------------
|   |   | O |
-------------
5 True
[['X', '', 'O'], ['', 'X', 'O'], ['', '', 'O']]


In [6]:
print('Attempt another move after game over:')
print(game.make_move(2, 0)) # returns 'F' for Forfeiture

# shouldn't have changed
game.display_board()

Attempt another move after game over:
F
-------------
| X |   | O |
-------------
|   | X | O |
-------------
|   |   | O |
-------------


In [7]:
print('Reset the game, display board, then attempt to play an illegal move:')

game.reset()
game.display_board()

print(game.make_move(3, 7)) # (3, 7) not valid cell coordinates, returns 'F' for Forfeiture
game.display_board() # should still be blank, as no legal move was played

Reset the game, display board, then attempt to play an illegal move:
-------------
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------
F
-------------
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------


In [138]:
# run this cell too. No need to read it unless you're interested or you think there's a bug

class NoughtsCrossesMatch:
    def __init__(self, noughts_player, crosses_player):
        self.players = {'O': noughts_player, 'X': crosses_player}
        self.game = NoughtsCrosses()
        self.results = []
    
    def request_move(self, symbol):
        '''
        ask a player what move it would like to make
        player *should* return a 2-tuple (row, col)
        '''
        game_state = self.game.board
        turn = self.game.turn
        player = self.players[symbol]
        try:
            return player.take_turn(game_state, turn)
        except: # if a player raises an exception, they forfeit
            self.declare_forfeit(symbol, 'failed to take_turn')
    
    def other_player(self, symbol):
        '''
        convenience function to convert one player's symbol into the other's
        '''
        return {'X': 'O', 'O': 'X'}[symbol]
            
    def declare_forfeit(self, forfeits, reason):
        '''
        forfeits must be one of 'O', 'X', 'both', 'XO', or 'OX' (the latter two will be converted to 'both')
        either calls .accept_forfeit() method of one player and .accept_win() of the other,
        or .accept_forfeit() of both
        appends result to self.results list,
        where result is 'F X reason' if only crosses forfeited, 'F O reason' if only noughts forfeited, and 'F both reason' if both forfeited
        Warning: this code is kinda ugly... :(
        '''
        if forfeits in ('XO', 'OX'):
            forfeits = 'both'
        if not forfeits in ('O', 'X', 'both'): # shouldn't ever happen...
            raise ValueError("forfeits must be one of 'O', 'X', 'both', 'XO', or 'OX'")
            
        if forfeits != 'both':
            winner_symbol = self.other_player(forfeits)
            winner = self.players[winner_symbol]
            try:
                winner.accept_win()
            except: # if a player fails to accept a win, they forfeit!
                new_reason = reason + ' and then winner failed to accept win'
                return self.declare_forfeit('both', new_reason)
            # the rest of this block only runs if winner has accepted the win
            forfeiter = self.players[forfeits]
            try:
                forfeiter.accept_forfeit()
            except:
                pass # this smells, but if a player fails to accept a forfeit, that's their problem!
        else:
            for symbol in ('O', 'X'):
                player = self.players[symbol]
                try:
                    player.accept_forfeit()
                except:
                    pass # this smells, but if a player fails to accept a forfeit, that's their problem!
        result = f'F {forfeits} {reason}'
        self.results.append(result)
    
    def declare_draw(self):
        '''
        calls .accept_draw() method of both players
        if either player fails to accept draw (by raising an exception), that player forfeits
        appends 'D' to self.results list
        '''
        forfeits = ''
        for symbol in self.players.keys():
            player = self.players[symbol]
            try:
                player.accept_draw()
            except:
                forfeits += symbol
        if forfeits != '':
            self.declare_forfeit(forfeits, 'failed to accept draw')
        else:
            self.results.append('D')
    
    def declare_winner(self, winning_symbol):
        '''
        calls .accept_win() on winner, .accept_loss() on loser
        if either player raises an exception, they forfeit
        appends winners symbol to self.results list
        '''
        winner = self.players[winning_symbol]
        losing_symbol = self.other_player(winning_symbol)
        loser = self.players[losing_symbol]
        forfeits = ''
        reason = ''
        try:
            winner.accept_win()
        except:
            forfeits += winning_symbol
            reason += f'{winning_symbol} failed to accept win'
        try:
            loser.accept_loss()
        except:
            forfeits += losing_symbol
            if reason != '':
                reason += ' '
            reason += f'{losing_symbol} failed to accept loss'
        if forfeits != '':
            self.declare_forfeit(forfeits, reason)
        else:
            result = winning_symbol
            self.results.append(result)
    
    def run_turn(self):
        '''
        request a move from player, try to run it, decide outcome (win, draw, forfeit, keep playing) and record
        '''
        player_symbol = ['O', 'X'][self.game.turn % 2] # even turns are noughts, odd are crosses
        move = self.request_move(player_symbol)
        if move == 'resign':
            winner = self.other_player(player_symbol)
            return self.declare_winner(winner) # return to prevent following code from executing in this case
        try:
            result = self.game.make_move(move[0], move[1])
        except:
            self.declare_forfeit(player_symbol, 'failed to return a legal format from take_turn')
            return 'F'
        if result == 'D':
            self.declare_draw()
        elif result in ('O', 'X'):
            self.declare_winner(result)
        elif result == 'F':
            self.declare_forfeit(player_symbol, 'played an illegal move')
        return result # will be 'O', 'X', 'D', 'F', or 'continue'
            
    def run_match(self):
        result = 'continue'
        while result == 'continue':
            result = self.run_turn()
        self.game.reset()
        return result
    
    def count_matching_results(self, string, exact=True, start=0, stop=None):
        '''
        counts the number of results in self.results matching the given string exactly or approximately
        exact matching should be used for detecting wins and draws, approximate matching for forfeits
        approximate matching only matches results STARTING with the specified string
        '''
        if stop == None:
            stop = len(self.results)
        def is_match(result):
            if exact:
                return result == string
            else:
                match_length = len(string)
                return result[:match_length] == string
        return len([result for result in self.results[start:stop] if is_match(result)])
    
    def get_statistics(self, start=0, stop=None):
        '''
        Get statistics on the results from game start (inclusive) to game stop (exclusive) (default: all games played)
        Returns a dictionary with keys 'O' (for wins by noughts player), 'X' (for wins by crosses player), 'D' (for draws),
        'F O' (for forfeits by noughts), 'F X' (for forfeits by crosses), and 'F both' (for double forfeitures)
        '''
        if stop == None:
            stop = len(self.results)
        stats = {}
        for key in ('O', 'X', 'D'):
            stats[key] = self.count_matching_results(key, True, start, stop)
        for key in ('F O', 'F X'):
            stats[key] = self.count_matching_results(key, False, start, stop)
        return stats
    
    def get_forfeiture_reasons(self):
        '''
        returns lists of reasons for forfeiture by player, assembled into a dict:
        {'X': [reasons X forfeited], 'O': [reasons O forfeited], 'b': [reasons both forfeited]}
        '''
        forfeitures = [result[2:] for result in self.results if result[0] == 'F'] # slice [2:] to remove the 'F ' at the start of each
        reasons = {}
        for player in ('X', 'O', 'b'):
            reasons[player] = [forfeit for forfeit in forfeitures if forfeit[0] == player]
        return reasons

<h3>
    Explanation of code:
</h3>
The NoughtsCrossesMatch class takes two players when instantiated; the first is the noughts player, the second is the crosses player. Note that these cannot change roles once the match is instantiated - you will probably want to create two NoughtsCrossMatch objects with the same two players in different roles, to allow them both to play both parts.

The main attribute of the class you will want to use is the results attribute, which is a list of strings recording the results of each game. The result can be 'X' if the crosses player won, 'O' if the noughts player won, 'D' for a draw, or a forfeiture string. The forfeiture strings are 'F O' if the noughts played forfeited, 'F X' if the crosses player forfeited, 'F both' if both have forfeited (this occurs if one player forfeits and then the other raises an error while the forfeit is processing, it should never happen in practice) and then the reason for the forfeiture. For instance, a forfeiture string might be 'F X played an illegal move', which will be recorded if the crosses player attempts to play in a non-empty cell, or to play in a cell outside the grid.

The main methods of the class you will want to use are run_match(), get_statistics, and get_forfeiture reasons. The run_match() method runs a complete game between the two players, records the result, and resets the game ready for another match. The get_statistics() method returns a dictionary of the results for certain games (specified by giving a start and stop argument; if these are omitted, the default is all games). The get_forfeiture_reasons() method returns a dictionary of the reasons for forfeiture of each player.

Aside from the results attribute and run_match(), get_statistics(), and get_forfeiture_reasons() methods, you should not need anything from this class - the rest of its code is for internal use.

In [ ]:
# do not attempt to run this cell at this point, it is simply to illustrate how you would use the class
# as there are no players yet, this cell will not run

# instantiate players - note: the NoughtsCrossesPlayer class has not yet been created, so this will produce an error if run at this point
player1 = NoughtsCrossesPlayer()
player2 = NoughtsCrossesPlayer()

match1 = NoughtsCrossesMatch(player1, player2) # player1 is Noughts, player two is Crosses
match2 = NoughtsCrossesMatch(player2, player1) # other way around, so both players get a chance to play both positions

# play 100 games
for i in range(100):
    match1.run_match()

# print the results of those hundred games
print(match1.results)
# sample output: ['X', 'D', 'F O played an illegal move', 'O', ...]

# print statistics of the hundred games
print(match1.get_statistics())
# sample output: {'X': 30, 'O': 10, 'D': 40, 'F O': 10, 'F X': 0, 'F both': 10}

# play 200 games the other way round
for i in range(200):
    match2.run_match()

# print statistics for just games 50 (incl) to 150 (excl)
print(match2.get_statistics(50, 150))
# same output format as previous
# note: you can use this way of limiting which games are included in the stats to compare results during the early games with results later on, once the players are trained

# print reasons for forfeitures
print(match1.get_forfeiture_reasons())
# sample output: {'O': ['O played an illegal move', ...], 'X': [...], 'b': [...]}

Finally, we look at the outline of a player class. Your task is to fill this in and develop a way for it to learn. Feel free to rename it and modify it, and work across multiple cells. Your player may be hard-coded with rules about the size of the board (<em>i.e.</em>, you can code it to only ever play in a cell that actually exists, not in cell (4,-9) or similar). You may not code it with any strategy, or that it may not play in a non-empty cell - it must learn these things for itself! Before any training, your code should play randomly in any cell, without regard to whether that cell is already full, or to winning or losing.

Your player class <strong>must</strong> contain the methods take_turn(game_state, turn), accept_win(), accept_loss(), accept_draw(), and accept_forfeit(). These are the methods which the NoughtsCrossesMatch uses to inteact with the player. It must also contain a name attribute to identify itself (if we have a tournament between players coded by several people, this will be used to identify who coded the winner!). You can define other attributes and/or methods, but these will not be used by the NoughtsCrossesMatch.

Instructions on the inputs and return values of these methods are in the cell below. The learning should happen when one of accept_win(), accept_loss(), accept_draw(), or accept_forfeit() is called (or you may keep an internal counter and run the learning after a certain number of games, if you prefer, but that will still need to be triggered within the call of one of those methods).

If your player is missing one of these required methods, or if the method has the wrong return type or inputs, it will raise an error when playing in the match, and will lose by forfeit.

Within this skeleton of a class is the method random_choice(probabilities), which takes a list of numbers and returns a random number from 0 to the length of the list, with probabilities in the ratio of the input list. You can use this method for selecting moves with specified probabilities, or you can write your own version, or look up ways of doing it. You can treat the method as a black box, or try to understand how it works.

In [25]:
import numpy as np # needed for the random_choice() method

class NoughtsCrossesPlayer:
    '''
    This is an outline of a class for a Noughts and Crosses playing code, with the methods it will need to interact with my other code.
    You can modify this class, define your own class inheriting from it, or define your own class from scratch. However, your code MUST have the methods specified below
    to interact correctly with the game. It may of course have additional methods. Each method has its own docstring describing expected behaviour.
    As currently written, the methods do nothing. You will need to fill in their details.
    '''
    def __init__(self, name):
        self.name = name # something that will let us identify who's it is for determining the winner of a match
        # you can add more things to __init__() if you want, or leave it at self.name
    
    def random_choice(self, probabilities):
        '''
        this method can be used to randomly select a move from possible moves with specified probabilities
        probabilities should be a list of positive ints or floats. This method returns an int from 0 (inclusive) to len(probabilities) (exclusive),
        where the probability of a number i being picked is proportional to the value of probabilities[i].
        E.g., random_choice([1,3]) has a 1/4 chance of returning 0 and a 3/4 chance of returning 1
        random_choice([2,0,7]) has a 2/9 chance of returning 0, no chance of returning 1, and a 7/9 chance of returning 2
        '''
        cumulative = np.array(probabilities).cumsum()
        low, high = 0, cumulative[-1]
        rand = np.random.default_rng() # random number generator
        sample = rand.uniform(low, high)
        return cumulative.searchsorted(sample)
    
    def take_turn(self, game_state, turn):
        '''
        This method should return a 2-tuple (row, column) of ints which are the (0-indexed) row and column numbers to play in, or the string 'resign' (case-sensitive!)
        
        game_state is a list of rows, where each row is a list of 3 strings, which can be '' (empty), 'X', or 'O'. It describes the current state of the board
        turn is an int, the turn number, passed for convenience. If turn is odd, it's Crosses turn to play; if turn is even, it's Noughts turn
        Note: turn is 0-indexed, so turn=0 when the board is empty
        
        E.g., for the first move of the game, take_turn([['','',''], ['','',''], ['','','']], 0) will be called. If it returns (0,2), that means play in the top right corner
        (2,0) would be bottom left, (1,1) is the middle cell, etc. If the method returns 'resign', the game ends and this player has lost.
        '''
        pass # your code here
    
    def accept_win(self):
        '''
        This method will be called to tell your code it has WON a game. It doesn't need to return anything, but can include your learning code, or a call to it
        '''
        pass
    
    def accept_loss(self):
        '''
        This method will be called to tell your code it has LOST a game. It doesn't need to return anything, but can include your learning code, or a call to it
        '''
        pass
    
    def accept_draw(self):
        '''
        This method will be called to tell your code a game has ended in a DRAW. It doesn't need to return anything, but can include your learning code, or a call to it
        '''
        pass
    
    def accept_forfeit(self):
        '''
        This method will be called to tell your code it has FORFEITED a game. It doesn't need to return anything, but can include your learning code, or a call to it
        A forfeit can be because your code has raised an error while trying to do something else, or because it has made an illegal move (e.g., tried to play in a cell that isn't empty)
        Note: if your code raises an error in its accept_win() method, it will forfeit. So you can even forfeit a game you had won!
        In win-loss statistics, a forfeiture will be considered as a loss; whether or not you treat it the same as a loss in the learning code is up to you
        '''
        pass

<h3>
    The tournament
</h3>
The code below will let you play different players against each other. You can use it to train two copies of your player against each other, and if multiple people attempt this challenge, we can use it to hold a tournament between them. I have coded one of these myself, so you can also play against mine.

In noughts-and-crosses, if both players play perfectly, the result is always a draw. Therefore any two successfully coded players should eventually always draw, once they have both learnt perfect play. So the tournament rules are as follows:
- any player which does not eventually reach a state of consistently drawing against other players ranks lower than one which does
- among players which eventually always draw against each other, the one which had the shorter training period before reaching that point ranks higher
So to win the tournament, you want your player to learn perfect play <em>as quickly as possible</em>.

Example usage is shown below.

In [174]:
# run this cell. Example usage is below

class NoughtsCrossesTournament:
    def __init__(self, player_list):
        '''
        player_list should be a list of *fresh, untrained* instances of NoughtsCrossesPlayer classes. Each instance *must* have a different .name attribute.
        The tournament will automatically pair off every instance against every other instance, both ways around - so each instance will play every other as noughts, and also as crosses.
        '''
        self.players = player_list.copy()
        self.rounds_played = 0
        
        # pair off each player
        self.matches = {}
        for i in range(len(self.players)):
            for j in range(len(self.players)):
                if i != j:
                    noughts_player = self.players[i]
                    crosses_player = self.players[j]
                    match_name = self.get_match_name_by_player(noughts_player, crosses_player)
                    self.matches[match_name] = NoughtsCrossesMatch(noughts_player, crosses_player)
        
    def get_match_name_by_player(self, noughts_player, crosses_player):
        '''
        convenience method to be called by other methods. Gives name of match between two players
        '''
        return f"O {noughts_player.name} - X {crosses_player.name}"
    
    def run_rounds(self, rounds=1):
        '''
        run a number of rounds (default 1) of the tournament.
        In a round, every match is played once, so every player plays every other player twice (once as O, once as X)
        '''
        for i in range(rounds):
            for match_name in self.matches:
                self.matches[match_name].run_match()
            self.rounds_played += 1
    
    def get_matches_by_player(self, player):
        '''
        returns dict of all matches involving a particular player, separated by noughts vs crosses
        '''
        noughts_matches = []
        crosses_matches = []
        # find all match names involving this player:
        for opponent in self.players:
            if opponent.name != player.name:
                player_as_noughts = self.get_match_name_by_player(player, opponent)
                player_as_crosses = self.get_match_name_by_player(opponent, player)
                noughts_match = self.matches[player_as_noughts]
                crosses_match = self.matches[player_as_crosses]
                noughts_matches.append(noughts_match)
                crosses_matches.append(crosses_match)
        return {'O': noughts_matches, 'X': crosses_matches}
    
    def get_stats_by_player(self, player, start=0, stop=None):
        '''
        gets aggregated statistics for all matches (from start to stop) played by a particular player
        gives both overall stats, and grouped by whether the player was noughts or crosses
        result will be a dictionary {'overall': overall_dict, 'O': O_dict, 'X': X_dict},
        where e.g., O_dict = {'O': 10, 'X': 15, 'D': 40, 'F O': 10, 'F X': 5}, similarly X_dict, and
        overall_dict is similar, but with keys 'wins', 'losses', 'draws', 'forfeits committed', 'forfeits received'
        '''
        # set up dictionary for stats breakdown
        stats_dict = {}
        for key in ('O', 'X'):
            stats_dict[key] = {'O': 0, 'X': 0, 'D': 0, 'F O': 0, 'F X': 0} # this is the format of the stats dictionary from NoughtsCrossesMatch.get_statistics()
        
        matches = self.get_matches_by_player(player)
        for symbol in ('O', 'X'):
            for match in matches[symbol]:
                match_stats = match.get_statistics(start, stop)
                for stats_key in match_stats:
                    stats_dict[symbol][stats_key] += match_stats[stats_key]
        wins = stats_dict['O']['O'] + stats_dict['X']['X']
        losses = stats_dict['O']['X'] + stats_dict['X']['O']
        draws = stats_dict['O']['D'] + stats_dict['X']['D']
        forfeits_committed = stats_dict['O']['F O'] + stats_dict['X']['F X']
        forfeits_received = stats_dict['O']['F X'] + stats_dict['X']['F O']
        stats_dict['overall'] = {'wins': wins,
                                 'losses': losses,
                                 'draws': draws,
                                 'forfeits committed': forfeits_committed,
                                 'forfeits_received': forfeits_received
                                }
        return stats_dict
    
    def get_forfeitures_by_player(self, player):
        '''
        returns lists of reasons for forfeiture by player, assembled into a dict:
        {'X': [reasons forfeited as X], 'O': [reasons forfeited as O], 'b': [reasons forfeited with opponent]}
        '''
        forfeitures = {'X': [], 'O': [], 'b': []}
        matches = self.get_matches_by_player(player)
        for symbol in matches:
            for match in matches[symbol]:
                reasons = match.get_forfeiture_reasons()
                forfeitures[symbol] += reasons[symbol]
                forfeitures['b'] += reasons['b']
        return forfeitures
    
    def reset(self):
        self.__init__(self.players)

<h3>
    Example usage
</h3>

In [ ]:
# do not run this cell until you have completed the NoughtsCrossesPlayer class

players = []
# generate 10 players, named 'player 0' to 'player 9':
for i in range(10):
    players.append(NoughtsCrossesPlayer(f'player {i}'))

# create tournament with these players
tournament = NoughtsCrossesTournament(players)

# run 100 rounds of the tournament
tournament.run_rounds(100)

# get tournament statistics:
for i in range(10):
    player = tournament.players[i]
    print(player.name)
    print(tournament.get_stats_by_player(player)) # can add additional start and stop arguments to get stats for certain games
    print('---------\n\n')